In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device(device_name)

Found GPU at: /device:GPU:0


# RNN --> SimpleRNN, GRU, LSTM

## Importing Libraries

In [2]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation, GRU, LSTM
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

## Loading Dataset
Different datasets are already present in keras, we are going to use one of them known as reuters.

The dataset is already preprcoessed and the words are already embeded and converted into numbers and we don't need to perform that operation. We will just load the data, perform some necessay operations like padding and one-hot encoding and then train and test our models. Don't worry if you didn't understand the terms, they will be explained ahead.

----

**About Data**

The data consits of sentences and we need to classify them.
The Sentences and target classed are already encoded as numbers and we can directly start with other necessary preprocessing steps like padding.


In [3]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [4]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2113536/2110848 [==============================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


## Data Preprocessing

### Padding
We will pass a sequence of words, i.e. a sentence encoded in the form of numbers into our model. Our model can only accept sentence of a predefined length thus we need to increase the lenght of short sentences. To perform this operation we will add 0's at the end of the sentence and make all of them of the same length. This is known as padding.

In [5]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

### Reshaping
We will now reshape our sentences from the shape of (none, 49) to (none, 49, 1), where none will be the number of sentences persent in respective dataset.

Inputs to Recurrent networks are 3 dimentional and are in the format of (batch_size, timesteps, input_dim), and thus we need to reshape them.
- **batch_size**: ommitted when creating RNN instance (== None). Usually designated when fitting model.
- **timesteps**: number of input sequence per batch
- **input_dim**: dimensionality of input sequence

In [6]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

### One-Hot Encoding
We will now ecode our target dataset using one hot encoding.
One hot encoding is the representation of the data in the from of an array of 0's and a single 1, the position of 1 denotes the original categorical value.

In [7]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

limit = y_train.shape[0]
y_train = y_data[:limit]
y_test = y_data[limit:]

## Model Building, Training and Testing

### SimpleRNN

Here we are going to use only one SimpleRNN layer and a Dense layer to make the model as simple as possible.
The model will classify the sentence to one of the 46 classes. 

In [8]:
# model building
def simple_rnn():
  model = Sequential()

  # return_sequence is set to true if the output of the current recurrent layer is to be fed to a next recurrent layer and false otherwise
  # this is because the recureent layers take sequence as input
  # if the next layer is a recurrent layer then it will also need a sequence of input
  # thus the current layer needs to return a sequence the next layer in recurrent
  model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = False))
  model.add(Dense(46))
  model.add(Activation('softmax'))
  
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
  
  return model

In [9]:
# model training
model = KerasClassifier(build_fn = simple_rnn, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 3s 35ms/step - loss: 3.5082 - accuracy: 0.2004
Epoch 2/200
28/28 [==============================] - 1s 36ms/step - loss: 1.5224 - accuracy: 0.7092
Epoch 3/200
28/28 [==============================] - 1s 35ms/step - loss: 1.2442 - accuracy: 0.6961
Epoch 4/200
28/28 [==============================] - 1s 35ms/step - loss: 1.2214 - accuracy: 0.6988
Epoch 5/200
28/28 [==============================] - 1s 35ms/step - loss: 1.1832 - accuracy: 0.7174
Epoch 6/200
28/28 [==============================] - 1s 35ms/step - loss: 1.1857 - accuracy: 0.7010
Epoch 7/200
28/28 [==============================] - 1s 35ms/step - loss: 1.2045 - accuracy: 0.6972
Epoch 8/200
28/28 [==============================] - 1s 37ms/step - loss: 1.1255 - accuracy: 0.7223
Epoch 9/200
28/28 [==============================] - 1s 36ms/step - loss: 1.1330 - accuracy: 0.7112
Epoch 10/200
28/28 [==============================] - 1s 36ms/step - loss: 1.1285 - accuracy: 0.7250

In [10]:
# model teting
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

simple_rnn_acc = accuracy_score(y_pred, y_test_) 
print(simple_rnn_acc)

12/12 [==============================] - 0s 4ms/step
0.7529215358931552


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


### GRU (Gated Recurrent Unit)

The model structure is same as above model, the only difference is that here we are using GRU layer instead of SimpleRNN layer.

In [11]:
# model building
def gru_rnn():
  model = Sequential()
  model.add(GRU(50, input_shape = (49,1), return_sequences = False))
  model.add(Dense(46))
  model.add(Activation('softmax'))
  
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
  
  return model

In [12]:
# model training
model = KerasClassifier(build_fn = gru_rnn, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 6s 6ms/step - loss: 3.6457 - accuracy: 0.5624
Epoch 2/200
28/28 [==============================] - 0s 5ms/step - loss: 2.0072 - accuracy: 0.7298
Epoch 3/200
28/28 [==============================] - 0s 5ms/step - loss: 1.2401 - accuracy: 0.7055
Epoch 4/200
28/28 [==============================] - 0s 5ms/step - loss: 1.0928 - accuracy: 0.7330
Epoch 5/200
28/28 [==============================] - 0s 5ms/step - loss: 1.1456 - accuracy: 0.7071
Epoch 6/200
28/28 [==============================] - 0s 5ms/step - loss: 1.1156 - accuracy: 0.7087
Epoch 7/200
28/28 [==============================] - 0s 5ms/step - loss: 1.0776 - accuracy: 0.6982
Epoch 8/200
28/28 [==============================] - 0s 5ms/step - loss: 1.0478 - accuracy: 0.7115
Epoch 9/200
28/28 [==============================] - 0s 5ms/step - loss: 0.9447 - accuracy: 0.7565
Epoch 10/200
28/28 [==============================] - 0s 5ms/step - loss: 0.8123 - accuracy: 0.7975
Epoch 11/

In [13]:
# model testing
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

gru_acc = accuracy_score(y_pred, y_test_) 
print(gru_acc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


12/12 [==============================] - 0s 2ms/step
0.8547579298831386


### LSTM (Long Short Term Memory)

The model structure is same as above models, the only difference is that here we are using LSTM layer instead of GRU or SimpleRNN layer.

In [14]:
# model building
def lstm_rnn():
  model = Sequential()
  model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
  model.add(Dense(46))
  model.add(Activation('softmax'))
  
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
  
  return model

In [15]:
# model training
model = KerasClassifier(build_fn = lstm_rnn, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 1s 6ms/step - loss: 3.4945 - accuracy: 0.4621
Epoch 2/200
28/28 [==============================] - 0s 6ms/step - loss: 1.4697 - accuracy: 0.7035
Epoch 3/200
28/28 [==============================] - 0s 5ms/step - loss: 1.1995 - accuracy: 0.7089
Epoch 4/200
28/28 [==============================] - 0s 5ms/step - loss: 1.1908 - accuracy: 0.7059
Epoch 5/200
28/28 [==============================] - 0s 5ms/step - loss: 1.1277 - accuracy: 0.7143
Epoch 6/200
28/28 [==============================] - 0s 5ms/step - loss: 1.0643 - accuracy: 0.7115
Epoch 7/200
28/28 [==============================] - 0s 5ms/step - loss: 0.9716 - accuracy: 0.7262
Epoch 8/200
28/28 [==============================] - 0s 5ms/step - loss: 0.9403 - accuracy: 0.7674
Epoch 9/200
28/28 [==============================] - 0s 5ms/step - loss: 0.8998 - accuracy: 0.7654
Epoch 10/200
28/28 [==============================] - 0s 5ms/step - loss: 0.8794 - accuracy: 0.7813
Epoch 11/

In [16]:
# model testing
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

lstm_acc = accuracy_score(y_pred, y_test_) 
print(lstm_acc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


12/12 [==============================] - 0s 2ms/step
0.8430717863105175


## Comparing Models

* We can see that GRU and LSTM perform better then simple RNN
* A GRU is slightly less complex but is approximately as good as an LSTM performance-wise when trained on small amount of data.
* LSTMs give better results than GRUs when trained on a sufficiently good amount of data.
---
GRUs are generally used when you do have long sequence training samples and you want a quick and decent accuracy and maybe in cases where infrastructure is an issue. LSTMs are preferred when sequence lengths are more and some good context is there.

In [17]:
print("Simple RNN Test Accuracy = ",simple_rnn_acc)
print("GRU Test Accuracy = ",gru_acc)
print("LSTM Test Accuracy = ",lstm_acc)

Simple RNN Test Accuracy =  0.7529215358931552
GRU Test Accuracy =  0.8547579298831386
LSTM Test Accuracy =  0.8430717863105175
